# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *


ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools


In [3]:
def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None):
    global colors
    global stuff
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()
    
    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        if 'TH' in hist.ClassName():
            hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options+'PA')
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()

    if v_lines:
        for v_line_x in v_lines:
            aline = ROOT.TLine(v_line_x, c.GetUymin(), v_line_x,  c.GetUymax())
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    if h_lines:
        for h_line_y in h_lines:
#             print h_line_y, c.GetUxmin(), c.GetUxmax()
            aline = ROOT.TLine(c.GetUxmin(), h_line_y, c.GetUxmax(),  h_line_y)
            aline.SetLineStyle(2)
            c.cd()
            aline.Draw("same")
            stuff.append(aline)
    c.Update()


In [2]:
# %load samples.py
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v80'


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0', 'PU0', 'v81', 'V9'),
    Sample('ele_flat2to100_PU200', 'PU200', 'v81', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V9')
    ]


gen_part_selections: 10


Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v80.root does not exist


In [3]:
# %load settings.py
# from samples import get_label_dict
import python.selections as selections
import python.collections as collections

import pprint



def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary



# === samples =====================================================
samples = []

samples = samples_ele_V8
samples += samples_ele_V9


sample = 'V9'

do_rate = False

# === TP ==========================================================
# tps = ['EG', 'TkEG', 'TkEle', 'TkIsoEle']
tps = ['DEF', 'HMvDR', 'HMvDRRebin', 'HMvDRsTC', 'HMvDRTrue']

# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['Em', 'all']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']
tp_select['HMvDRRebin'] = tp_select['DEF']
tp_select['HMvDRsTC'] = tp_select['DEF']
tp_select['HMvDRTrue'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC', 'GENEtaD']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']



event_manager = collections.EventManager()

all_tpsets = {}
for collection in event_manager.collections:
    all_tpsets[collection.name] = collection.label


# all_tpsets = {'DEF': 'dRC3d',
#               'DEFCalib': 'NNDR Calib v1',
#               'DEFNC': 'dRC3d + new Th',
#               'HMvDR': 'HistoMaxC3d + dR(layer)',
#               'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
#               'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
#               'EG': 'EG',
#               'TkEG': 'TkEG',
#               'TkEle': 'TkEle',
#               'TkIsoEle': 'TkIsoEle',
#               'L1Trk': 'L1Track'}


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


[EventManager] registering collection: MC
[EventManager] registering collection: GEN
[EventManager] registering collection: TC
[EventManager] registering collection: TCTrue
[EventManager] registering collection: DEF2D
[EventManager] registering collection: DEF2DTrue
[EventManager] registering collection: HMvDRTrue
[EventManager] registering collection: DEF
[EventManager] registering collection: DEFNC
[EventManager] registering collection: HMvDR
[EventManager] registering collection: HMvDRRebin
[EventManager] registering collection: HMvDRsTC
[EventManager] registering collection: HMvDRNC0
[EventManager] registering collection: HMvDRNC1
[EventManager] registering collection: DEFMerged
[EventManager] registering collection: DEFCalib
[EventManager] registering collection: HMvDRMerged
[EventManager] registering collection: TT
[EventManager] registering collection: SimTT
[EventManager] registering collection: HgcrocTT
[EventManager] registering collection: WaferTT
[EventManager] registering 

In [7]:
tpsets

{'DEF': 'dRC3d',
 'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDRRebin': 'HM+dR(layer) rebin Cl3d ',
 'HMvDRTrue': 'HM+dR(layer) True Cl3d',
 'HMvDRsTC': 'HM+dR(layer) SuperTC Cl3d '}

In [4]:
samples

[<drawingTools.Sample instance at 0x11a09f200>,
 <drawingTools.Sample instance at 0x11a09fc68>,
 <drawingTools.Sample instance at 0x11a09fc20>,
 <drawingTools.Sample instance at 0x11a09fd40>]

In [5]:
tpset_selections

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'all': ''}

In [6]:
gen_selections

{'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [7]:
import python.selections as selections
selections.gen_part_selections
selections.gen_part_selections_debug

[<Selection n: GENEtaBCD, s: (((abs(pdgid) == 11) & (firstmother_pdgid == 11)) & (reachedEE == 2)) & (1.52 < abs(eta) <= 2.8), l:1.52 < |#eta^{GEN}| <= 2.8> ]

In [8]:
from python.plotters import tp_genmatched_plotters

histo_primitives = pd.DataFrame() 

for plotter in tp_genmatched_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
print histo_primitives.data.unique()

['DEF' 'HMvDRTrue' 'HMvDR' 'HMvDRRebin' 'HMvDRsTC']


In [5]:
%%time
# for key in samples_ele[0].histo_file.GetListOfKeys():
#     #pass
#     key.GetName()
# print len(samples_ele[0].histo_file.GetListOfKeys())
# l = [histo for histo in samples_ele[0].histo_file.GetListOfKeys()]
l = [(histo.ReadObj(), histo.GetName()) for histo in samples_ele[0].histo_file.GetListOfKeys() if 'DEF_all'+'_' in histo.GetName()]

CPU times: user 12 ms, sys: 7.34 ms, total: 19.4 ms
Wall time: 21.3 ms


In [9]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)


for tp in tps:
    print tp
    print histo_primitives[histo_primitives.data == tp].data_sel.unique()
    print gen_selections
    print samples
    hplot.cache_histo(classtype=histos.HistoSetEff, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=gen_selections)



DEF
['all' 'Pt10' 'Pt20' 'Pt30' 'Em' 'EmPt10' 'EmPt20' 'EmPt30']
{'GENPt40': 'p_{T}^{GEN}>=40GeV', 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4', 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7', 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8', 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4', 'GENPt30': 'p_{T}^{GEN}>=30GeV', 'GENPt20': 'p_{T}^{GEN}>=20GeV', 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8', 'GENPt10': 'p_{T}^{GEN}>=10GeV', 'nomatch': '', 'GEN': ''}
[<drawingTools.Sample instance at 0x11a09f200>, <drawingTools.Sample instance at 0x11a09fc68>, <drawingTools.Sample instance at 0x11a09fc20>, <drawingTools.Sample instance at 0x11a09fd40>]
<drawingTools.Sample instance at 0x11a09f200>
<drawingTools.Sample instance at 0x11a09f200> DEF all GENPt40
<drawingTools.Sample instance at 0x11a09f200> DEF all GENEtaC
<drawingTools.Sample instance at 0x11a09f200> DEF all GENEtaB
<drawingTools.Sample instance at 0x11a09f200> DEF all GENEtaD
<drawingTools.Sample instance at 0x11a09f200> DEF all GENEtaBC
<drawingTools.Sample ins

<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENPt40
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENEtaC
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENEtaB
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENEtaD
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENEtaBC
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENPt30
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENPt20
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENEtaBCD
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GENPt10
<drawingTools.Sample instance at 0x11a09fc68> DEF Em nomatch
<drawingTools.Sample instance at 0x11a09fc68> DEF Em GEN
<drawingTools.Sample instance at 0x11a09fc68> DEF EmPt10 GENPt40
<drawingTools.Sample instance at 0x11a09fc68> DEF EmPt10 GENEtaC
<drawingTools.Sample instance at 0x11a09fc68> DEF EmPt10 GENEtaB
<drawingTools.Sample instance at 0x11a09fc68> DEF EmPt10 GENEtaD
<drawingTools.Sample instance at 0x11a09fc68> DEF EmPt10 GENEtaBC
<dra

<drawingTools.Sample instance at 0x11a09fd40> DEF all GENPt30
<drawingTools.Sample instance at 0x11a09fd40> DEF all GENPt20
<drawingTools.Sample instance at 0x11a09fd40> DEF all GENEtaBCD
<drawingTools.Sample instance at 0x11a09fd40> DEF all GENPt10
<drawingTools.Sample instance at 0x11a09fd40> DEF all nomatch
<drawingTools.Sample instance at 0x11a09fd40> DEF all GEN
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENPt40
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENEtaC
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENEtaB
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENEtaD
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENPt30
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENPt20
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 GENPt10
<drawingTools.Sample instance at 0x11a09fd40> DEF Pt10 nomat

<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt20 GENPt30
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt20 GENPt20
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt20 GENEtaBCD
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt20 GENPt10
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt20 nomatch
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt20 GEN
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENPt40
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENEtaC
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENEtaB
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENEtaD
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENEtaBC
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENPt30
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENPt20
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GENEtaBCD
<drawingTools.Sample instance at 0x11a09f200> HMvDR EmPt30 GE

<drawingTools.Sample instance at 0x11a09fc20> HMvDR Pt30 GENPt10
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Pt30 nomatch
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Pt30 GEN
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENPt40
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENEtaC
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENEtaB
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENEtaD
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENEtaBC
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENPt30
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENPt20
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENEtaBCD
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GENPt10
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em nomatch
<drawingTools.Sample instance at 0x11a09fc20> HMvDR Em GEN
<drawingTools.Sample instance at 0x11a09fc20> HMvDR EmPt10 GENPt40
<drawingTools.Sample instance at 0x11a09fc20> HMvD

...

<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC Em GENPt10
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC Em nomatch
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC Em GEN
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENPt40
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENEtaC
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENEtaB
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENEtaD
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENEtaBC
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENPt30
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENPt20
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENEtaBCD
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GENPt10
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 nomatch
<drawingTools.Sample instance at 0x11a09f200> HMvDRsTC EmPt10 GEN
<drawingTools.Sample instance at 0x11

<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt10 GENPt30
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt10 GENPt20
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt10 GENPt10
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt10 nomatch
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt10 GEN
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt20 GENPt40
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt20 GENEtaC
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt20 GENEtaB
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt20 GENEtaD
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt20 GENEtaBC
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt20 GENPt30
<drawingTools.Sample instance at 0x11a09fc20> HMvDRsTC Pt20 GENPt20
<drawingTools.Sample instance at 0x11a09fc20> HM

<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt20 GENEtaBC
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt20 GENPt30
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt20 GENPt20
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt20 GENEtaBCD
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt20 GENPt10
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt20 nomatch
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt20 GEN
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt30 GENPt40
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt30 GENEtaC
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt30 GENEtaB
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt30 GENEtaD
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt30 GENEtaBC
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt30 GENPt30
<drawingTools.Sample instance at 0x11a09fd40> HMvDRsTC EmPt30 GENPt20
<drawingTools.Sample

<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt10 GENEtaBC
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt10 GENPt30
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt10 GENPt20
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt10 GENPt10
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt10 nomatch
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt10 GEN
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt20 GENPt40
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt20 GENEtaC
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt20 GENEtaB
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt20 GENEtaD
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt20 GENEtaBC
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt20 GENPt30
<drawingTools.Sample instance at 0x11a09fc68> HMvDRTrue Pt20 GENPt20
<drawingTools.Sample instance at 0

<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt20 GENPt20
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt20 GENEtaBCD
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt20 GENPt10
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt20 nomatch
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt20 GEN
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENPt40
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENEtaC
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENEtaB
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENEtaD
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENEtaBC
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENPt30
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENPt20
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENEtaBCD
<drawingTools.Sample instance at 0x11a09fc20> HMvDRTrue EmPt30 GENPt10
<draw

In [7]:
%%time

pt_points = ['Pt10', 'Pt20', 'Pt30']


for pu in ['PU0', 'PU200']:
    for tp in ['DEF', 'HMvDR']:
        print 'TP: {}'.format(tp)
        for tp_sel in ['EmPt10', 'EmPt20', 'EmPt30']:
            tp_sel_den = 'all'
            if 'Pt' in tp_sel:
                # print tp_sel
                tp_sel_den = tp_sel.split('Pt')[0]
                if tp_sel_den == '':
                    tp_sel_den = 'all'
                print '{} den -> {}'.format(tp_sel, tp_sel_den) 
            else:
                continue
            for gen_sel in ['GENEtaBC']:
                hsetden = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel_den, gen_sel)
                hset = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, gen_sel)
#                 print hset[0][0].name
#                 print hsetden[0][0].name
                hset[0][0].computeTurnOn(hsetden[0][0].h_num)


TP: DEF
EmPt10 den -> Em
EmPt20 den -> Em
EmPt30 den -> Em
TP: HMvDR
EmPt10 den -> Em
EmPt20 den -> Em
EmPt30 den -> Em
TP: DEF
EmPt10 den -> Em
EmPt20 den -> Em
EmPt30 den -> Em
TP: HMvDR
EmPt10 den -> Em
EmPt20 den -> Em
EmPt30 den -> Em
CPU times: user 5.3 s, sys: 975 ms, total: 6.27 s
Wall time: 6.41 s


In [11]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V8,PU0,DEF,all,GENPt40,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5d4bd8>
1,V8,PU0,DEF,all,GENEtaC,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5cd9e0>
2,V8,PU0,DEF,all,GENEtaB,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d588bd8>
3,V8,PU0,DEF,all,GENEtaD,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d588a70>
4,V8,PU0,DEF,all,GENEtaBC,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5cdb48>
5,V8,PU0,DEF,all,GENPt30,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5cdf38>
6,V8,PU0,DEF,all,GENPt20,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5c4680>
7,V8,PU0,DEF,all,GENEtaBCD,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5c4128>
8,V8,PU0,DEF,all,GENPt10,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5cde60>
9,V8,PU0,DEF,all,nomatch,python.l1THistos.HistoSetEff,<drawingTools.HProxy instance at 0x11d5ca170>


In [12]:
hplot.data.tp.unique()

array(['DEF', 'HMvDR', 'HMvDRRebin', 'HMvDRsTC', 'HMvDRTrue'], dtype=object)

In [13]:
hplot.data.gen_sel.unique()

array(['GENPt40', 'GENEtaC', 'GENEtaB', 'GENEtaD', 'GENEtaBC', 'GENPt30',
       'GENPt20', 'GENEtaBCD', 'GENPt10', 'nomatch', 'GEN'], dtype=object)

In [14]:
hplot.data['sample'].unique()

array(['V8', 'V9'], dtype=object)

## Effinciency vs $\eta$

In [16]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
#             print [his.h_num.h_eta.GetName() for his in hsets]

In [24]:
for tp in tps:
    for tp_sel in ['Pt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU0'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')


In [23]:
for tp in tps:
    for tp_sel in ['EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU0'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0.75, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')
     

In [18]:
for tp in tps:
    for tp_sel in ['EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, None, ['PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, 
                     text=text, min_y=0, max_y=1.1, 
                     v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8],
                     y_axis_label='(L1>thr. & matched to GEN)/GEN',
                     options='P')
     

In [13]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['all', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tps, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

In [14]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Pt20', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', pu, 'HMvDR', tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

In [15]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Pt20', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['V9'], pu, ['HMvDR', 'HMvDRNC0'], tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Pt20', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets_new = []
            labels_new = []
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'V8', pu, 'HMvDR', tp_sel, gen_sel)
            hsets_new.extend(hsets)
            labels_new.append('HM 3D (V8 geom.)')
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, ['V9'], pu, ['HMvDR', 'HMvDRNC0'], tp_sel, gen_sel)
            hsets_new.extend(hsets)
            labels_new.append('HM 3D (V9 geom.)')
            labels_new.append('HM 3D (V9 geom. + tuned FE th.)')

            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets_new], labels_new, text=text, min_y=0.75, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['Pt20', 'EmPt20']:
        for gen_sel in ['GENPt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tps, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_eta.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, v_lines=[-2.8, -2.4, -1.7, -1.52, 1.52, 1.7, 2.4, 2.8])
   

## Efficiency vs $p_{T}$

In [ ]:
for tp in tps:
    for tp_sel in ['all', 'Em']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1] )


In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in tps:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, ['all', 'Em'], gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


In [ ]:
for pu in ['PU0', 'PU200']:
    for tp_sel in ['all','Em']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tps, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_eff.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN/GEN', h_lines=[0.95, 1])


## Turn-on curves

In [18]:
for tp in tps:
    for tp_sel in ['EmPt20','EmPt30']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
    for tp in tps:
        for tp_sel in ['Pt20', 'EmPt20', 'Pt30', 'EmPt30']:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, ['GENEtaABC', 'GENEtaBC'])
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'EmPt30']:
            for gen_sel in gen_select:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tps, tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                draw([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFCalib'], tp_sel, ['GENEtaABC', 'GENEtaBC'])
            # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
            drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

In [ ]:
for pu in ['PU0', 'PU200']:
        for tp_sel in ['EmPt20', 'Emv1Pt20', 'EmPt30', 'Emv1Pt30']:
            for gen_sel in gen_select:
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, ['DEF', 'DEFMerged'], tp_sel, gen_sel)
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                drawSame([his.h_ton.h_pt.CreateGraph() for his in hsets], labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
   

## Scaling

In [19]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(3, 0.8, 1)
tf_yc.SetParLimits(0, 0, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [22]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []


for pu in ['PU0', 'PU200']:
    for tp in ['DEF']:
        for pt in [10, 20, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, sample, pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                draw(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

------- PU0, dRC3d, EGId, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, dRC3d, EGId, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU0, dRC3d, EGId, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, dRC3d, EGId, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, dRC3d, EGId, p_{T}^{L1}>=20GeV, 1.52 < |#eta^{GEN}| <= 2.4
None
------- PU200, dRC3d, EGId, p_{T}^{L1}>=30GeV, 1.52 < |#eta^{GEN}| <= 2.4
None

****************************************
Minimizer is Minuit / MigradImproved
Chi2                      =      444.254
NDf                       =           36
Edm                       =  2.91996e-10
NCalls                    =          607
#lambda                   =     0.247816   +/-   0.060618     	 (limited)
#mu                       =      3.46141   +/-   0.900639    
#sigma                    =      6.90811   +/-   0.771622    
Plateau                   =     0.996239   +/-   0.000902273  	 (limited)
Baseline         

In [ ]:

import numpy as np
from array import array

def f_cb(x, par):
    return ROOT.Math.crystalball_function(x[0], par[0], par[1], par[2], par[3])

tf_cb = ROOT.TF1('cb', f_cb, 0, 100, 4)
tf_cb.SetParameters(10., 4., 3, 20.)

def f_cbi(x, par):
    return ROOT.Math.crystalball_cdf(x[0], par[0], par[1], par[2], par[3])
#     tf_cb.SetParameters(par[0], par[1], par[2], par[3])
#     npts = 1000
#     x_ = []
#     w_ = []
   
#     return tf_cb.IntegralFast(npts, np.array(npts), np.array(npts), 0., x[0])
    #return tf_cb.Integral(0., x[0])
    
    
tf_cbi = ROOT.TF1('cbi', f_cbi, 0, 100, 4)
tf_cbi.SetParameters(10., 4., 3, 20.)
tf_cbi.SetLineColor(4)

c = newCanvas()
tf_cbi.Draw()
c.Draw()


    # ROOT::Math::crystalball_function (double x, double alpha, double n, double sigma, double x0=0)

# fitf_cb = ROOT.TF1('cbfunc', 'ROOT.TMath.crystalball_function (x, [0], [1], [2], [3])', 0, 100)
# 

# fitf_cb.SetParameters(1, 1, 1, 0)


In [ ]:
# Double_t turnon_func(Double_t *x, Double_t *par)
# {
#   double halfpoint = par[0];
#   double slope = par[1];
#   double plateau = par[2];

#   //double offset = par[3];
#   //double plateau = 1.0;
#   double offset = 0;

#   double pt = TMath::Max(x[0],0.000001);

#    double arg = 0;
#    //cout << pt <<", "<< halfpoint <<", " << slope <<endl;
#    arg = (pt - halfpoint)/(TMath::Sqrt(pt)*slope);
#    double fitval = offset+0.5*plateau*(1+TMath::Erf(arg));
#    return fitval;
# }




def turnon_func1(x, par):

    return 0.5 * par[0] * (1+ROOT.TMath.Erf((x[0]-par[1])/(par[2]*ROOT.TMath.Sqrt(2))))




def turnon_func(x, par):
  
    halfpoint = par[0]
    slope = par[1]
    plateau = par[2]
    offset = 0
    pt = ROOT.TMath.Max(x[0],0.000001)
    arg = 0
    arg = (pt - halfpoint)/(ROOT.TMath.Sqrt(pt)*slope)
    fitval = offset+0.5*plateau*(1+ROOT.TMath.Erf(arg))
    return fitval

fitf_to = ROOT.TF1('myfunc', turnon_func1, 0, 100, 3)
#fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf_to.SetParameters(1,25,2)
fitf_to.SetLineColor(3)

def fermi(x, params):
    return params[3]*(params[0] - 1/(ROOT.TMath.Exp((x[0]-params[1])/(params[2]))+1))
  
fitf = ROOT.TF1('myfunc', fermi, 0, 100, 4)
fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf.SetParameters(1,25,2,1)



c = newCanvas()
fitf.Draw()
c.Draw()


In [ ]:
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []


for pu in ['PU0', 'PU200']:
    for tp in ['DEF', 'DEFCalib', 'DEFMerged']:
        for pt in [10, 20, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, 'GENEtaBC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

In [ ]:
points

In [ ]:
# print points

points_calib_pu0 = [point for point in points if ('Calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'Calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if (point[0]=='DEF' and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if (point[0]=='DEF' and 'PU200' in point[1])]

points_merged_pu0 = [point for point in points if (point[0]=='DEFMerged' and 'PU0' in point[1])]
points_merged_pu200 = [point for point in points if (point[0]=='DEFMerged' and 'PU200' in point[1])]

# print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

def drawScaling(points, label, text):
    graph =  ROOT.TGraph(len(points), array('d', [point[2] for point in points]), array('d',[point[3] for point in points]))
    drawGraphsSame([graph], labels=[label], min_y=0, max_y=50, options='A*', text=text)
    graph.Fit('pol1', 'S')
    
    ROOT.gStyle.SetOptFit(11111)
    return graph

g1 = drawScaling(points=points_nocalib_pu0, label='PU0', text='PU0: no calib')
g3 = drawScaling(points=points_calib_pu0, label='PU0', text='PU0: calib')
g5 = drawScaling(points=points_merged_pu0, label='PU0', text='PU0: merged')

g2 = drawScaling(points=points_nocalib_pu200, label='PU200', text='PU200: no calib')
g4 = drawScaling(points=points_calib_pu200, label='PU200', text='PU200: calib')
g6 = drawScaling(points=points_merged_pu200, label='PU200', text='PU200: merged')


# graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
# drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
# graph_nocalib_pu0.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

# graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
# drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
# graph_calib_pu0.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)


# graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
# drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
# graph_nocalib_pu200.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

# graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
# drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
# graph_calib_pu200.Fit('pol1', 'S')
# ROOT.gStyle.SetOptFit(11111)

In [ ]:
points = []


for pu in ['PU0', 'PU200']:
    for tp in ['DEF', 'DEFCalib']:
        for pt in [10, 20, 25, 30]:
                tp_sel = 'EmPt{}'.format(pt)
                hsets, labels, text = hplot.get_histo(histos.HistoSetEff, 'ele', pu, tp, tp_sel, 'GENEtaABC')
                # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
                h_eff_vs_pt = [his.h_ton.h_pt.CreateGraph() for his in hsets]
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                print '------- {}'.format( text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                points.append((tp, pu, pt, pt095))
                drawSame(h_eff_vs_pt, labels, text=text, min_y=0, max_y=1.1, y_axis_label='L1 matched to GEN and p_{T}>th./L1 matched to GEN', h_lines=[0.95, 1])
                ROOT.gStyle.SetOptFit(11111)

In [ ]:
points_calib_pu0 = [point for point in points if ('Calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'Calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if ('Calib' not in point[0] and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if ( 'Calib' not in point[0] and 'PU200' in point[1])]

print points
# print points_calib_pu0
# print points_calib_pu200
# print points_nocalib_pu0
# print points_nocalib_pu200

graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)


graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

In [ ]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD', 'eleAAA', 'eleAA', 'eleBB', 'eleCC', 'eleDD']


labels = [sample.label for sample in samples]




histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# print histo_num_df
for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_df



In [ ]:
titles_df

In [ ]:
dir(histo_df.ele['DEF'][0])


In [ ]:
histo_df.loc['DEF', 'ele']

In [ ]:

for particle in ['ele']:
    for tp_set in tp_sets:
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.ele[tp_set]]
        drawGraphsSame(h_eff_vs_eta, labels, text=titles_df.loc[tp_set, particle])



In [ ]:
line095 = ROOT.TLine(0,0.95,110,0.95)
line095.SetLineStyle(2)
for particle in particles:
    for tp_set in tp_sets:
        #print 'particle: {}, TP set: {}, text: {}'.format(particle, tp_set, titles_df.loc[tp_set, particle])
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histo_df.loc[tp_set, particle]]
        drawGraphsSame(h_eff_vs_pt, labels, text=titles_df.loc[tp_set, particle], min_y=0., max_y=1.1)
        line095.Draw('same')

In [ ]:
histo_den_df

In [ ]:
# build turnon

for particle in particles:
    TP_set_den = 'DEF' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20', 'DEF_pt25', 'DEF_pt30'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

In [ ]:
for particle in particles:
    TP_set_den = 'DEF_em' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

# Photon pt35 GeV 1.6 < |eta| < 2.8 

In [ ]:

samples = samples_photon
tp_sets = ['DEF', 'DEF_em']
particles = ['photon']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        histo_df



In [ ]:
dir(histo_df.photon['DEF'][0])


In [ ]:
tp_set = 'DEF'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)

tp_set = 'DEF_em'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)
